In [2]:
%pip install markov_clustering[drawing]

  Using cached markov_clustering-0.0.6.dev0-py3-none-any.whl (6.3 kB)


You should consider upgrading via the 'c:\users\krister\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [98]:
import pandas as pd
import numpy as np
import networkx as nx
from itertools import combinations
from tqdm.notebook import tqdm
import markov_clustering as mc

# Clustering

## 1. Based on correlations of speed

In [ ]:
main_df = pd.read_csv('../data/processed_data.tsv', sep='\t', encoding='utf8')

In [4]:
df = pd.read_csv('../streamlit/streamlit_data.tsv', sep='\t', encoding='utf8', index_col='Unnamed: 0')
#df = df.replace(np.nan, pd.NA)

In [5]:
df = df.loc[:, df.columns[:50]].copy()

In [6]:
df

,Paris,London,Berlin,St. Petersburg,Wien,Madrid,Konstantinopel,Rom,Brüssel,Kopenhagen,...,Kiel,Leipzig,Frankfurt am Main,Köln,Italien,Washington,Königsberg,Darmstadt,Mailand,Breslau
1802,11.614583,11.294118,7.272727,11.131868,12.814815,NaN,35.214286,27.500000,10.666667,NaN,...,NaN,NaN,10.000000,NaN,21.333333,NaN,10.000,6.000000,20.666667,NaN
1803,10.600000,13.115789,9.428571,12.490385,12.562500,30.250000,34.384615,25.000000,9.884615,8.000000,...,NaN,NaN,9.000000,11.000000,21.857143,NaN,19.000,9.000000,22.200000,NaN
1804,11.669231,13.900000,6.000000,14.387387,13.837838,30.428571,39.210526,25.428571,9.666667,7.764706,...,NaN,9.000000,9.400000,NaN,19.545455,NaN,12.000,9.000000,21.500000,NaN
1805,11.794872,13.814815,7.972222,12.349057,13.716981,26.222222,41.045455,29.111111,9.000000,7.083333,...,NaN,10.846154,10.000000,NaN,20.846154,NaN,NaN,NaN,21.000000,11.333333
1806,12.914286,14.392405,8.513514,14.574074,16.580000,34.333333,39.550000,32.714286,NaN,9.411765,...,NaN,14.375000,24.444444,NaN,26.500000,NaN,16.750,NaN,25.000000,20.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1884,2.249344,1.808511,1.825301,1.726415,2.062500,2.000000,1.526316,1.852941,1.916667,1.777778,...,3.833333,2.428571,NaN,2.000000,NaN,2.269231,NaN,3.333333,NaN,2.000000
1885,2.312704,1.791096,1.681818,2.189781,1.937500,2.552941,1.350000,2.500000,4.028571,1.897436,...,2.000000,2.800000,3.000000,2.000000,NaN,1.555556,2.000,1.800000,NaN,2.250000
1886,2.324627,1.832727,1.754386,1.836842,2.071713,2.135135,1.481481,2.456140,2.425000,1.823529,...,1.666667,2.888889,NaN,2.142857,NaN,2.833333,2.500,1.500000,NaN,2.285714
1887,2.059211,1.961207,1.677922,1.682540,1.927711,2.409091,1.941176,1.960938,2.364865,1.833333,...,2.000000,2.375000,NaN,2.250000,NaN,2.333333,3.000,1.800000,1.750000,3.000000


In [178]:
corr = df.corr()
corr.fillna(0, inplace=True)

In [179]:
corr[corr < 0.85] = 0

In [180]:
corr.style.background_gradient(cmap ='coolwarm')

,Paris,London,Berlin,St. Petersburg,Wien,Madrid,Konstantinopel,Rom,Brüssel,Kopenhagen,New York,München,Riga,Hamburg,Turin,Bern,Dorpat,Dresden,Reval,Stockholm,Warschau,Domesnees,Florenz,Odessa,Neapel,Haag,Amsterdam,Athen,Lissabon,Hannover,Stuttgart,Bukarest,Triest,Versailles,Moskau,Karlsruhe,Main,Kassel,Belgrad,Budapest,Kiel,Leipzig,Frankfurt am Main,Köln,Italien,Washington,Königsberg,Darmstadt,Mailand,Breslau
Paris,1.000000,0.915437,0.000000,0.000000,0.855081,0.000000,0.000000,0.000000,0.860532,0.000000,0.000000,0.852811,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.874058,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.962795,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
London,0.915437,1.000000,0.000000,0.000000,0.905222,0.000000,0.000000,0.000000,0.925377,0.000000,0.000000,0.863551,0.000000,0.000000,0.000000,0.868471,0.000000,0.000000,0.000000,0.878462,0.000000,0.000000,0.000000,0.000000,0.000000,0.859793,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.859235,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.982090,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Berlin,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.923758,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.920570,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
St. Petersburg,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.877841,0.000000,0.000000,0.000000,0.000000,0.000000,0.865594,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.936603,0.000000,0.000000,0.000000,0.000000,0.000000,0.886041,0.000000,0.000000,0.000000,0.000000
Wien,0.855081,0.905222,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.891442,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.898411,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.887492,0.000000,0.000000,0.000000,0.853519,0.000000,0.000000,0.902303,0.000000,0.000000,0.000000,0.000000,0.000000,0.853964,0.000000,0.000000,0.000000,0.000000
Madrid,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.929876,0.952520,0.000000,0.000000,0.867747,0.000000,0.000000,0.000000,0.889488,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.912671,0.000000,0.905720,0.000000,0.000000,0.000000,0.930981,0.000000,0.000000,0.852003,0.857067,0.000000,0.000000,0.000000,0.000000,0.000000,0.937721,0.973497,0.000000,0.000000,0.000000,0.000000,0.000000,0.906499,0.000000,0.000000,0.898409,0.000000
Konstantinopel,0.000000,0.000000,0.000000,0.000000,0.000000,0.929876,1.000000,0.944078,0.000000,0.000000,0.950021,0.000000,0.000000,0.000000,0.862064,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.887738,0.000000,0.936556,0.000000,0.000000,0.880472,0.951825,0.000000,0.000000,0.914593,0.857204,0.000000,0.000000,0.000000,0.000000,0.000000,0.931736,0.969009,0.000000,0.000000,0.000000,0.000000,0.000000,0.946231,0.000000,0.000000,0.894768,0.000000
Rom,0.000000,0.000000,0.000000,0.000000,0.000000,0.952520,0.944078,1.000000,0.000000,0.000000,0.904475,0.000000,0.000000,0.000000,0.885869,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.923805,0.000000,0.924732,0.000000,0.000000,0.911403,0.931116,0.000000,0.000000,0.871835,0.898119,0.000000,0.000000,0.000000,0.000000,0.000000,0.860327,0.986226,0.000000,0.

In [181]:
G = nx.Graph()

In [182]:
G.add_nodes_from(corr.index)

In [183]:
for u, v in list(combinations(corr.index, 2)):
    if corr.loc[u, v] > 0:
        G.add_edge(u, v, weight=corr.loc[u, v])

In [184]:
matrix = nx.to_scipy_sparse_matrix(G)

In [187]:
result = mc.run_mcl(matrix, inflation=10, expansion=2)
clusters = mc.get_clusters(result)
                           
for cluster in clusters:
    places = [corr.index[ix] for ix in cluster]
    print(places)

['Paris', 'London']
['Berlin', 'Domesnees']
['St. Petersburg']
['Wien', 'Brüssel', 'München', 'Bern', 'Dorpat', 'Reval', 'Main', 'Budapest', 'Italien']
['Madrid', 'Konstantinopel', 'Rom', 'New York', 'Turin', 'Neapel', 'Lissabon', 'Bukarest', 'Belgrad']
['Kopenhagen']
['Riga']
['Hamburg']
['Dresden']
['Stockholm', 'Triest']
['Warschau']
['Florenz']
['Odessa']
['Haag', 'Amsterdam']
['Athen']
['Hannover']
['Stuttgart']
['Versailles']
['Moskau']
['Karlsruhe']
['Kassel']
['Kiel']
['Leipzig']
['Frankfurt am Main']
['Köln']
['Washington']
['Königsberg']
['Darmstadt']
['Mailand']
['Breslau']


## 2. Based on dates and speed

In [195]:
date_codes = {date: num for num, date in enumerate(main_df.doc_date.unique())}
main_df['date_code'] = [date_codes[date] for date in main_df['doc_date']]

In [197]:
y_train = df.columns[:50]

In [203]:
def generate_date_speed_array(main_df, place):
    
    array_df = main_df[main_df.placename.isin(df.columns[:50])].copy()

    dates = array_df.date_code[array_df.placename == place].reindex(array_df.index)
    speeds = array_df.delta[array_df.placename == place].reindex(array_df.index)
    return pd.concat([dates, speeds]).fillna(0).values

In [205]:
x_train = np.array([generate_date_speed_array(main_df, place) for place in y_train])

In [206]:
x_train.shape

(50, 389610)

In [202]:
main_df[main_df.placename.isin(df.columns[:50])]

,doc_id,doc_date,placename,day,day2,month,month2,origin_year,start,end,origin_date,delta,date_code
0,0,1802-01-01,St. Petersburg,20,NaN,12,NaN,NaN,0,32,1801-12-20,12,0
1,1,1802-01-01,Paris,18,NaN,12,NaN,NaN,0,27,1801-12-18,14,0
2,2,1802-01-01,Haag,22,NaN,12,NaN,NaN,0,27,1801-12-22,10,0
4,4,1802-01-01,Wien,19,NaN,12,NaN,NaN,0,26,1801-12-19,13,0
5,5,1802-01-01,München,19,NaN,12,NaN,NaN,0,24,1801-12-19,13,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
246630,289703,1888-12-31,London,10,29.0,1,12.0,NaN,1517,1552,1888-12-29,2,18476
246631,289703,1888-12-31,Belgrad,10,29.0,1,12.0,NaN,1751,1787,1888-12-29,2,18476
246632,289703,1888-12-31,New York,9,28.0,1,12.0,NaN,2009,2044,1888-12-28,3,18476
246634,289704,1888-12-31,St. Petersburg,31,NaN,12,NaN,NaN,1560,1586,1888-12-31,0,18476


In [91]:
main_df

,doc_id,doc_date,placename,day,day2,month,month2,origin_year,start,end,origin_date,delta
0,0,1802-01-01,St. Petersburg,20,NaN,12,NaN,NaN,0,32,1801-12-20,12
1,1,1802-01-01,Paris,18,NaN,12,NaN,NaN,0,27,1801-12-18,14
2,2,1802-01-01,Haag,22,NaN,12,NaN,NaN,0,27,1801-12-22,10
3,3,1802-01-01,Zürich,16,NaN,12,NaN,NaN,0,28,1801-12-16,16
4,4,1802-01-01,Wien,19,NaN,12,NaN,NaN,0,26,1801-12-19,13
...,...,...,...,...,...,...,...,...,...,...,...,...
246632,289703,1888-12-31,New York,9,28.0,1,12.0,NaN,2009,2044,1888-12-28,3
246633,289704,1888-12-31,PeterSbnrs,31,NaN,12,NaN,NaN,0,26,1888-12-31,0
246634,289704,1888-12-31,St. Petersburg,31,NaN,12,NaN,NaN,1560,1586,1888-12-31,0
246635,289704,1888-12-31,Berlin,11,NaN,1,NaN,NaN,2559,2578,1888-01-11,355


In [97]:
main_df[main_df.placename.isin(df.columns[:50])].groupby('doc_date')['placename'].unique()

doc_date
1802-01-01    [St. Petersburg, Paris, Haag, Wien, München, F...
1802-01-08    [Stockholm, Bern, Berlin, Paris, Haag, Wien, M...
1802-01-11                                    [Stockholm, Main]
1802-01-15                             [Stockholm, Paris, Haag]
1802-01-18    [St. Petersburg, London, Paris, Konstantinopel...
                                    ...                        
1888-12-27                           [Riga, Berlin, Paris, Rom]
1888-12-28    [Riga, Leipzig, Berlin, London, Belgrad, St. P...
1888-12-29                               [Riga, Berlin, Neapel]
1888-12-30    [Riga, Dorpat, Berlin, St. Petersburg, Bern, M...
1888-12-31    [Berlin, Kopenhagen, London, Belgrad, New York...
Name: placename, Length: 18468, dtype: object

In [116]:
def count_cooccurrences(main_df, placenames):
    
    df = pd.DataFrame(0, index=placenames, columns=placenames)
    
    for issue in tqdm(main_df[main_df.placename.isin(df.columns[:50])].groupby('doc_date')['placename'].unique()):
        for place1, place2 in combinations(issue, 2):
            df.loc[place1, place2] += 1
            df.loc[place2, place1] += 1
            
    return df

occ_df = count_cooccurrences(main_df, df.columns[:50])

  0%|          | 0/18468 [00:00<?, ?it/s]

In [121]:
len(main_df.placename[main_df.placename == 'London'])

21928

In [123]:
def weigh_occurrences(main_df, occ_df):
    
    df = occ_df.copy()
    
    for place1, place2 in tqdm(combinations(occ_df.index, 2)):
        total_occurrences = len(main_df[main_df.placename == place1]) + len(main_df.placename == place2)
        df.loc[place1, place2] = df.loc[place1, place2]/total_occurrences
        df.loc[place2, place1] = df.loc[place2, place1]/total_occurrences
        
    return df

weighed_df = weigh_occurrences(main_df, occ_df)

0it [00:00, ?it/s]

In [130]:
weighed_df.Moskau.sort_values()

Moskau               0.000000
Italien              0.000040
Mailand              0.000073
Frankfurt am Main    0.000077
Main                 0.000089
Darmstadt            0.000093
Breslau              0.000097
Königsberg           0.000105
Triest               0.000133
Kassel               0.000141
Leipzig              0.000145
Lissabon             0.000153
Haag                 0.000157
Köln                 0.000157
Washington           0.000170
Karlsruhe            0.000170
Kiel                 0.000170
Hannover             0.000177
Neapel               0.000198
Stuttgart            0.000202
Dresden              0.000246
Stockholm            0.000254
Turin                0.000281
Belgrad              0.000283
Warschau             0.000290
Florenz              0.000294
Athen                0.000323
Versailles           0.000339
Budapest             0.000351
Bern                 0.000354
Amsterdam            0.000371
Bukarest             0.000375
Odessa               0.000423
Dorpat    